# Spark Preparation
We check if we are in Google Colab.  If this is the case, install all necessary packages.

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 3.2.1 with hadoop 3.2, Java 8 and Findspark to locate the spark in the system. The tools installation can be carried out inside the Jupyter Notebook of the Colab.
Learn more from [A Must-Read Guide on How to Work with PySpark on Google Colab for Data Scientists!](https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/)

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
if IN_COLAB:
    !apt-get install openjdk-8-jdk-headless -qq > /dev/null
    !wget https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
    !tar xf spark-3.3.1-bin-hadoop2.tgz
    !mv spark-3.3.1-bin-hadoop2 spark
    !pip install -q findspark


--2022-11-05 06:20:15--  https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 274099817 (261M) [application/x-gzip]
Saving to: ‘spark-3.3.1-bin-hadoop2.tgz’

spark-3.3.1-bin-had 100%[===================>] 261.40M   255MB/s    in 1.0s    

2022-11-05 06:20:17 (255 MB/s) - ‘spark-3.3.1-bin-hadoop2.tgz’ saved [274099817/274099817]



In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark"

In [ ]:
# # innstall java
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null

# # install spark (change the version number if needed)
# !wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz

# # unzip the spark file to the current folder
# !tar xf spark-3.0.0-bin-hadoop3.2.tgz

# # set your spark folder to your system path environment. 
# import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"


# # install findspark using pip
# !pip install -q findspark

^C

gzip: stdin: unexpected end of file
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now


# Start a Local Cluster

In [ ]:
import findspark
findspark.init()

In [ ]:
spark_url = 'local'

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [ ]:
spark = SparkSession.builder\
        .master(spark_url)\
        .appName('Spark SQL')\
        .getOrCreate()

# Spark Assignment

Based on the movie review dataset in 'netflix-rotten-tomatoes-metacritic-imdb.csv', answer the below questions.

**Note:** do not clean or remove missing data

In [ ]:
path = '/content/drive/MyDrive/data_engineerclass/netflix-rotten-tomatoes-metacritic-imdb.csv'
df = spark.read.option("delimiter", ",").option("header", True).csv(path)

In [ ]:
df.show(5)

+-------------------+--------------------+--------------------+----------------+---------------+----------------+--------------------+------------+---------------+--------------------+--------------------+-----------+----------+---------------------+----------------+---------------+--------------------+----------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+------------+
|              Title|               Genre|                Tags|       Languages|Series or Movie|Hidden Gem Score|Country Availability|     Runtime|       Director|              Writer|              Actors|View Rating|IMDb Score|Rotten Tomatoes Score|Metacritic Score|Awards Received|Awards Nominated For| Boxoffice|Release Date|Netflix Release Date|    Production House|        Netflix Link|           IMDb Link|             Summary|IMDb Votes|               Image|              

In [ ]:
df.columns
cols = [c.replace(' ', '_') for c in df.columns]
df = df.toDF(*cols)

In [ ]:
df.columns

['Title',
 'Genre',
 'Tags',
 'Languages',
 'Series_or_Movie',
 'Hidden_Gem_Score',
 'Country_Availability',
 'Runtime',
 'Director',
 'Writer',
 'Actors',
 'View_Rating',
 'IMDb_Score',
 'Rotten_Tomatoes_Score',
 'Metacritic_Score',
 'Awards_Received',
 'Awards_Nominated_For',
 'Boxoffice',
 'Release_Date',
 'Netflix_Release_Date',
 'Production_House',
 'Netflix_Link',
 'IMDb_Link',
 'Summary',
 'IMDb_Votes',
 'Image',
 'Poster',
 'TMDb_Trailer',
 'Trailer_Site']

In [ ]:
df.show(5)

+-------------------+--------------------+--------------------+----------------+---------------+----------------+--------------------+------------+---------------+--------------------+--------------------+-----------+----------+---------------------+----------------+---------------+--------------------+----------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+------------+
|              Title|               Genre|                Tags|       Languages|Series_or_Movie|Hidden_Gem_Score|Country_Availability|     Runtime|       Director|              Writer|              Actors|View_Rating|IMDb_Score|Rotten_Tomatoes_Score|Metacritic_Score|Awards_Received|Awards_Nominated_For| Boxoffice|Release_Date|Netflix_Release_Date|    Production_House|        Netflix_Link|           IMDb_Link|             Summary|IMDb_Votes|               Image|              

## What is the maximum and average of the overall hidden gem score?

In [ ]:
from pyspark.sql.functions import avg, min, max, countDistinct, count, split,explode

In [ ]:
#Answer
df.filter(df.Hidden_Gem_Score >= 0).select(max('Hidden_Gem_Score'),avg('Hidden_Gem_Score')).show()

+---------------------+---------------------+
|max(Hidden_Gem_Score)|avg(Hidden_Gem_Score)|
+---------------------+---------------------+
|                  9.8|    5.937551386501226|
+---------------------+---------------------+



## How many movies that are available in Korea?

In [ ]:
#Answer
df.filter("Series_or_Movie == 'Movie'").where("Country_Availability like '%Korea%'").count()

3075

## Which director has the highest average hidden gem score?

In [ ]:
#Answer
df.groupby('Director').agg({'Hidden_Gem_Score': 'max'}).sort(col("max(Hidden_Gem_Score)").desc()).show()

+--------------------+---------------------+
|            Director|max(Hidden_Gem_Score)|
+--------------------+---------------------+
|         Dorin Marcu|                  9.8|
|                null|                  9.7|
|    Fernando Escovar|                  9.6|
|Julia Reichert, S...|                  9.6|
|          Rosa Russo|                  9.5|
|    Ignacio Busquier|                  9.5|
|         Kate Brooks|                  9.5|
|Vincent Bal, Kenn...|                  9.5|
|Bill Butler, Will...|                  9.5|
|     Charles Officer|                  9.4|
|    Ashish R. Shukla|                  9.3|
|         Lisa France|                  9.3|
|      Aundre Johnson|                  9.3|
|          Oh Jin-Koo|                  9.3|
|           Ryan Sage|                  9.3|
|   Frederico Machado|                  9.3|
|Jacqui Morris, Da...|                  9.3|
|        R.J. Bentler|                  9.3|
|        Shinkyu Choi|                  9.3|
|    Jan P

## How many genres are there in the dataset?

In [ ]:
#Answer
new_df = df.groupBy("Genre").agg(count('Genre'))
new_df.withColumn('Genre',explode(split('Genre',', '))).select(countDistinct("Genre")).show()

+---------------------+
|count(DISTINCT Genre)|
+---------------------+
|                   28|
+---------------------+

